In [1]:
!pip install lightkurve

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.9/888.9 kB 79.5 MB/s eta 0:00:00
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11373 sha256=e7989347f8e64c1ceb34d300ee3795c217ce2c3234412046137289478199b262
  Stored in directory: /root/.cache/pip/wheels/3c/ea/60/8d1c9fbbc99492a1775b36a5e29c8c1ef309cc5821bd5a219d
  Created wheel for memoization: filename=memoization-0.4.0-py3-none-any.whl size=50453 sha256=5a551f6f3b4d66bea10e79e26c0adc8e40153871f185fdef7a27857a5092771b
  Stored in directory: /root/.cache/pip/wheels/3e/b8/c5/b55

In [3]:
import csv
import json
import lightkurve as lk
import numpy as np
import numpy.ma as ma

##############################################################################################################
# For given number of star systems in the target file download LC data, save it into JSON/text file
##############################################################################################################

def download_LC_data(numSystems,targetStarNameFile, targetSaveFile):

  #get List of systems to examine from target File, read them into array Systems_To_Download
  Systems_to_download = []
  with open(targetStarNameFile,newline='') as csvFile:
    reader = csv.DictReader(csvFile)

    for row in reader:
      Systems_to_download.append(row)

  #Download data, append to dict
  for X in range(numSystems):
    currentDict = Systems_to_download[X]

    #download Lightkurve search object
    search_result = lk.search_lightcurve(currentDict["star_name"])

    #download data set from star (just one of several for now)
    data = search_result.download()

    #Errors are horizontal bars, messy, going to ignore them for now
    data = data.remove_nans()

    #get the actaul data from the LC object. This is stored in a masked array,
    #i want it in an array of floats so I can save it an not have to download it
    #again
    fluxValsMasked = data.flux.value

    LC_Data = []
    for dataPoint in fluxValsMasked:
      LC_Data.append(float(dataPoint))
      #print(type(dataPoint), dataPoint)



    #add LC data to the current dictionary obj so we can save it.
    currentDict['starData'] = LC_Data[:]

  FirstDict = Systems_to_download[0]

  #save all elements as json elements
  with open(targetSaveFile, 'w') as fout:

    for x in range(numSystems):
      currentDict = Systems_to_download[x]
      json.dump(currentDict, fout)
      fout.write("\n")

    #for dic in Systems_to_download:
    #  json.dump(dic, fout)
    #  fout.write("\n")

#FilteredExoplanetSystems
#Gaia_LC_Systems_0_2k.csv

download_LC_data(1000, 'Gaia_LC_Systems_0_2k.csv', 'NonPlanSystems1k.txt')

#Note to self: try this https://www.geeksforgeeks.org/write-a-dictionary-to-a-file-in-python/

/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 5 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 11 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 12 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 13 f

In [4]:
import csv
import json
import lightkurve as lk
import numpy as np
import numpy.ma as ma
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding

GaiaSystemNames = []
ExoplanetarySystems = []
FilteredGaiaSystems = []

TrainingSystems = []
TrainingYValues = np.array([])
TestSystems = []
TestYValues = np.array([])

Training_LC_Data = np.ndarray([])
Test_LC_Data = np.ndarray([])

maxLen = 0

####################################################################################
# Grab data from our JSON arrays, new for this version of this program
####################################################################################
def ReadStarDataFromFile(targetFileName):
  DataArray = []
  with open(targetFileName) as filereader:
    for row in filereader:
      currentData = json.loads(row)
      DataArray.append(currentData)
      #print(currentData['star_name'], currentData['starData'])
  return DataArray

##############################################################################################################
#  randomly generate test/training groups from the star systems we have identified as candidates.
# We mix systems we know have exoplanets with systems that don't have anything confirmed.
##############################################################################################################
def CreateGroups(NumTrainingSystems, NumTestSystems):
  NegativeGroup = FilteredGaiaSystems[:]
  PositiveGroup = ExoplanetarySystems[:]

  CountPositive = 0
  CountNegative = 0

  global TrainingYValues
  global TestYValues

  #create traning candidate list
  for x in range(NumTrainingSystems):
    WhichGroup = random.randint(0, 1)

    if(WhichGroup) == 0:
      NegativeTarget = random.randint(0, len(NegativeGroup)-1)

      CurrentDict = NegativeGroup.pop(NegativeTarget)
      #print("TestPrint",CurrentDict)

      TrainingSystems.append(CurrentDict)
      #TrainingYValues.append(0)

      TrainingYValues = np.append(TrainingYValues,0)

      CountNegative = CountNegative + 1
    else:
      PositiveTarget = random.randint(0, len(PositiveGroup)-1)

      CurrentDict = PositiveGroup.pop(PositiveTarget)
      #print("TestPrint2",CurrentDict)

      TrainingSystems.append(CurrentDict)
      TrainingYValues = np.append(TrainingYValues,1)

      CountPositive = CountPositive + 1

  print("Training Distribution:",CountPositive,CountNegative)

  CountPositive = 0
  CountNegative = 0

  #create test candidate list
  for x in range(NumTestSystems):
    WhichGroup = random.randint(0, 1)

    if(WhichGroup) == 0:
      NegativeTarget = random.randint(0, len(NegativeGroup)-1)
      CurrentDict = NegativeGroup.pop(NegativeTarget)
      TestSystems.append(CurrentDict)


      TestYValues = np.append(TestYValues,0)
      #TestYValues.append(0)
      CountNegative = CountNegative + 1
    else:
      PositiveTarget = random.randint(0, len(PositiveGroup)-1)
      CurrentDict = PositiveGroup.pop(PositiveTarget)
      TestSystems.append(CurrentDict)

      TestYValues = np.append(TestYValues,1)
      #TestYValues.append(1)
      CountPositive = CountPositive + 1
  #print("Test Distribution:",CountPositive,CountNegative)

##############################################################################################################
#  For the candidate systems we have identified and put into lists we download the lightcurve data
# Also get maxLen (biggest length of LC data we have) since that is important for the AI model
# We padd the Data using a Keras function since they all need to have the same width
##############################################################################################################
def GetLightKurveData():
  global Training_LC_Data
  global Test_LC_Data
  global maxLen
  tempTrainingList = []
  TempTestList = []

  for x in TrainingSystems:
    data = x['starData']
    tempTrainingList.append(data)
    if len(data)> maxLen:
      maxLen = len(data)
  for y in TestSystems:
    data = y['starData']
    TempTestList.append(data)
    if len(data)> maxLen:
      maxLen = len(data)

  #since there is zero garantee that the observations from any two stars will have the same number of data entries
  #we need to padd them with zeros. An embedded mask layer will then work in the AI model to ignore this zeroed data
  #https://www.tensorflow.org/guide/keras/understanding_masking_and_padding

  Training_LC_Data = tf.keras.utils.pad_sequences(tempTrainingList, maxlen = maxLen, dtype='float32', padding='post')
  Test_LC_Data = tf.keras.utils.pad_sequences(TempTestList, maxlen = maxLen, dtype='float32', padding='post')


##############################################################################################################
#   The AI model doing the actual work. This is set up to work on Binary Classification.
##############################################################################################################

def RunAI():

  #https://www.atmosera.com/blog/binary-classification-with-neural-networks/
  model = Sequential()
  # layers.Embedding(input_dim=maxLen, output_dim=64, mask_zero=True),
  #model.add(Embedding(input_dim=maxLen, output_dim=maxLen, mask_zero=True))
  model.add(Dense(2120, activation='relu', input_dim=maxLen))
  model.add(Dense(1060, activation='relu', input_dim=2120))
  model.add(Dense(512, activation='relu', input_dim=1060))
  model.add(Dense(254, activation='relu', input_dim=512))
  model.add(Dense(124, activation='relu', input_dim=254))
  model.add(Dense(62, activation='relu', input_dim=124))
  model.add(Dense(32, activation='relu', input_dim=62))
  model.add(Dense(16, activation='relu', input_dim=32))
  model.add(Dense(8, activation='relu', input_dim=16))
  model.add(Dense(4, activation='relu', input_dim=8))
  model.add(Dense(2, activation='relu', input_dim=4))
  model.add(Dense(1, activation='sigmoid'))
  #adam
  model.compile(optimizer='Adadelta', loss='binary_crossentropy', metrics=['accuracy'])
  model.build()
  model.summary()

  model.fit(Training_LC_Data, TrainingYValues, batch_size=1, epochs=15, validation_split=0.15)

  score = model.evaluate(Test_LC_Data,TestYValues, verbose=0)



ExoplanetarySystems = ReadStarDataFromFile('ExoPlanSystems.txt')
FilteredGaiaSystems = ReadStarDataFromFile('NonPlanSystems.txt')
CreateGroups(200,25)
GetLightKurveData()

print("Number of Exoplanet Systems:",len(ExoplanetarySystems))
print("Number of Non-Exoplanet Systems:",len(FilteredGaiaSystems))


print("Length Training Data:",len(TrainingSystems), len(TrainingYValues))
print("Length Training Y Values",len(TestSystems),  len(TestYValues))

RunAI()

Training Distribution: 103 97
Number of Exoplanet Systems: 500
Number of Non-Exoplanet Systems: 500
Length Training Data: 200 200
Length Training Y Values 25 25
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 2120)              228898520 
                                                                 
 dense_13 (Dense)            (None, 1060)              2248260   
                                                                 
 dense_14 (Dense)            (None, 512)               543232    
                                                                 
 dense_15 (Dense)            (None, 254)               130302    
                                                                 
 dense_16 (Dense)            (None, 124)               31620     
                                                                 
 dense_17 (Dense)        

In [11]:
import csv
import json
import lightkurve as lk
import numpy as np
import numpy.ma as ma
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding

GaiaSystemNames = []
ExoplanetarySystems = []
FilteredGaiaSystems = []

TrainingSystems = []
TrainingYValues = np.array([])
TestSystems = []
TestYValues = np.array([])

Training_LC_Data = np.ndarray([])
Test_LC_Data = np.ndarray([])

maxLen = 0

####################################################################################
# Grab data from our JSON arrays, new for this version of this program
####################################################################################
def ReadStarDataFromFile(targetFileName):
  DataArray = []
  with open(targetFileName) as filereader:
    for row in filereader:
      currentData = json.loads(row)
      DataArray.append(currentData)
      #print(currentData['star_name'], currentData['starData'])
  return DataArray

##############################################################################################################
#  randomly generate test/training groups from the star systems we have identified as candidates.
# We mix systems we know have exoplanets with systems that don't have anything confirmed.
##############################################################################################################
def CreateGroups(NumTrainingSystems, NumTestSystems):
  NegativeGroup = FilteredGaiaSystems[:]
  PositiveGroup = ExoplanetarySystems[:]

  CountPositive = 0
  CountNegative = 0

  global TrainingYValues
  global TestYValues

  #create traning candidate list
  for x in range(NumTrainingSystems):
    WhichGroup = random.randint(0, 1)

    if(WhichGroup) == 0:
      NegativeTarget = random.randint(0, len(NegativeGroup)-1)

      CurrentDict = NegativeGroup.pop(NegativeTarget)
      #print("TestPrint",CurrentDict)

      TrainingSystems.append(CurrentDict)
      #TrainingYValues.append(0)

      TrainingYValues = np.append(TrainingYValues,0)

      CountNegative = CountNegative + 1
    else:
      PositiveTarget = random.randint(0, len(PositiveGroup)-1)

      CurrentDict = PositiveGroup.pop(PositiveTarget)
      #print("TestPrint2",CurrentDict)

      TrainingSystems.append(CurrentDict)
      TrainingYValues = np.append(TrainingYValues,1)

      CountPositive = CountPositive + 1

  print("Training Distribution:",CountPositive,CountNegative)

  CountPositive = 0
  CountNegative = 0

  #create test candidate list
  for x in range(NumTestSystems):
    WhichGroup = random.randint(0, 1)

    if(WhichGroup) == 0:
      NegativeTarget = random.randint(0, len(NegativeGroup)-1)
      CurrentDict = NegativeGroup.pop(NegativeTarget)
      TestSystems.append(CurrentDict)


      TestYValues = np.append(TestYValues,0)
      #TestYValues.append(0)
      CountNegative = CountNegative + 1
    else:
      PositiveTarget = random.randint(0, len(PositiveGroup)-1)
      CurrentDict = PositiveGroup.pop(PositiveTarget)
      TestSystems.append(CurrentDict)

      TestYValues = np.append(TestYValues,1)
      #TestYValues.append(1)
      CountPositive = CountPositive + 1
  #print("Test Distribution:",CountPositive,CountNegative)

##############################################################################################################
#  For the candidate systems we have identified and put into lists we download the lightcurve data
# Also get maxLen (biggest length of LC data we have) since that is important for the AI model
# We padd the Data using a Keras function since they all need to have the same width
##############################################################################################################
def GetLightKurveData():
  global Training_LC_Data
  global Test_LC_Data
  global maxLen
  tempTrainingList = []
  TempTestList = []

  for x in TrainingSystems:
    data = x['starData']
    tempTrainingList.append(data)
    if len(data)> maxLen:
      maxLen = len(data)
  for y in TestSystems:
    data = y['starData']
    TempTestList.append(data)
    if len(data)> maxLen:
      maxLen = len(data)

  #since there is zero garantee that the observations from any two stars will have the same number of data entries
  #we need to padd them with zeros. An embedded mask layer will then work in the AI model to ignore this zeroed data
  #https://www.tensorflow.org/guide/keras/understanding_masking_and_padding

  Training_LC_Data = tf.keras.utils.pad_sequences(tempTrainingList, maxlen = maxLen, dtype='float32', padding='post')
  Test_LC_Data = tf.keras.utils.pad_sequences(TempTestList, maxlen = maxLen, dtype='float32', padding='post')


##############################################################################################################
#   The AI model doing the actual work. This is set up to work on Binary Classification.
##############################################################################################################

def RunAI():
  #https://www.atmosera.com/blog/binary-classification-with-neural-networks/
  model = Sequential()
  # layers.Embedding(input_dim=maxLen, output_dim=64, mask_zero=True),
  #model.add(Embedding(input_dim=maxLen, output_dim=maxLen, mask_zero=True))
  model.add(Dense(2600, activation='relu', input_dim=maxLen))
  model.add(Dense(312, activation='relu', input_dim=2600))
  model.add(Dense(124, activation='relu', input_dim=312))
  model.add(Dense(16, activation='relu', input_dim=124))
  model.add(Dense(2, activation='relu', input_dim=16))
  model.add(Dense(1, activation='sigmoid'))
  #adam
  model.compile(optimizer='Adadelta', loss='binary_crossentropy', metrics=['accuracy'])
  model.build()
  model.summary()

  model.fit(Training_LC_Data, TrainingYValues, batch_size=10, epochs=40, validation_split=0.10)

  score = model.evaluate(Test_LC_Data,TestYValues, verbose=0)



ExoplanetarySystems = ReadStarDataFromFile('ExoPlanSystems500.txt')
FilteredGaiaSystems = ReadStarDataFromFile('NonPlanSystems500.txt')
CreateGroups(800,50)
GetLightKurveData()

print("Number of Exoplanet Systems:",len(ExoplanetarySystems))
print("Number of Non-Exoplanet Systems:",len(FilteredGaiaSystems))

print("Max width of data", maxLen)

print("Length Training Data:",len(TrainingSystems), len(TrainingYValues))
print("Length Training Y Values",len(TestSystems),  len(TestYValues))

RunAI()

Training Distribution: 441 359
Number of Exoplanet Systems: 500
Number of Non-Exoplanet Systems: 500
Max width of data 117688
Length Training Data: 800 800
Length Training Y Values 50 50
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 2600)              305991400 
                                                                 
 dense_11 (Dense)            (None, 312)               811512    
                                                                 
 dense_12 (Dense)            (None, 124)               38812     
                                                                 
 dense_13 (Dense)            (None, 16)                2000      
                                                                 
 dense_14 (Dense)            (None, 2)                 34        
                                                                 